# Analysis of the School Budget Data

Author: Tom Freudenmann
Date: 2024-17-01
Tags: Analysis, School, Budget, Data
Summary: This is the analysis of the school budgets in Germany.


In [3]:
from school_analysis.preprocessing.load import Loader
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import school_analysis as sa
from school_analysis.preprocessing.helpers import students_teachers
from tueplots.constants.color import rgb
from school_analysis.analysis.exploration import Exploration
from school_analysis.analysis.aggregation import Aggregation
from school_analysis.plotting.general import GeneralPlots

FEDERAL_STATES= {
    "Old Federal States": ["Schleswig-Holstein", "Niedersachsen", "Bremen", "Hamburg", "Nordrhein-Westfalen", "Hessen", "Rheinland-Pfalz", "Saarland", "Baden-Württemberg", "Bayern"],
    "New Federal States": ["Mecklenburg-Vorpommern", "Brandenburg", "Berlin", "Sachsen", "Sachsen-Anhalt", "Thüringen"]
}

loader = Loader()

budgets_per_child = loader.load("budgets-per-child-by-state")
budgets = loader.load("budgets-by-state")
verbraucherpreisindex = loader.load("verbraucherpreisindex-state")

In [4]:
display(budgets_per_child, budgets, verbraucherpreisindex)

,Federal State,Year,Budget
0,Baden-Württemberg,2010,6100
1,Bayern,2010,6400
2,Berlin,2010,7000
3,Brandenburg,2010,6200
4,Bremen,2010,6100
...,...,...,...
199,Sachsen,2021,8700
200,Sachsen-Anhalt,2021,8600
201,Schleswig-Holstein,2021,8600
202,Thüringen,2021,9500


,Institution,Year,Measure,Unit,Value,Federal State
0,Child day care,1995,Public expenditure on education,EUR 1000,242182.09,Baden-Württemberg
1,Child day care,1995,Public expenditure on education,EUR 1000,377243.42,Bayern
2,Child day care,1995,Public expenditure on education,EUR 1000,NaN,Berlin
3,Child day care,1995,Public expenditure on education,EUR 1000,206027.11,Brandenburg
4,Child day care,1995,Public expenditure on education,EUR 1000,48907.11,Bremen
...,...,...,...,...,...,...
39899,Total,2023,Prop. of ed. exp. of pub.budgets of overall bu...,percent,24.32,Saarland
39900,Total,2023,Prop. of ed. exp. of pub.budgets of overall bu...,percent,27.83,Sachsen
39901,Total,2023,Prop. of ed. exp. of pub.budgets of overall bu...,percent,25.89,Sachsen-Anhalt
39902,Total,2023,Prop. of ed. exp. of pub.budgets of overall bu...,percent,25.58,Schleswig-Holstein


,Year,Federal State,Index,Year Relative
0,1995,Baden-Württemberg,0.705,2020
1,1996,Baden-Württemberg,0.713,2020
2,1997,Baden-Württemberg,0.725,2020
3,1998,Baden-Württemberg,0.732,2020
4,1999,Baden-Württemberg,0.736,2020
...,...,...,...,...
459,2019,Thüringen,0.988,2020
460,2020,Thüringen,1.000,2020
461,2021,Thüringen,1.032,2020
462,2022,Thüringen,1.109,2020
